In [45]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

## The panelists by country

Reading in the panel information Netquest gave us.

### may need to be set

In [46]:
paneldate = "190628"

### needs oversight

In [47]:
outdir = "../out/panel_country/"

In [48]:
paneldir = ("C:/Users/schadem/Box Sync/LAPOP Shared/working documents/maita/Coordination/"+
            "IDB Online Trust/raw/netquest/"+paneldate+"/")
panelfile = [s for s in os.listdir(paneldir) if s.endswith(".csv")][0]
print("loading panel data: ", panelfile)


loading panel data:  VANDUS_1147438_w2_20190628.csv


### this can just do its thing

In [49]:
panels = pd.read_csv(paneldir+panelfile,
                     sep=';', 
#                      skiprows=2891450,
#                      nrows=1000000,
                     na_values=[' ','.'], 
                     encoding='latin1',
                     #error_bad_lines=False
                    )

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (169) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Creating a dictionary of individual country dataframes, and cleaning them up

In [50]:
panels_dict = {}
for p in panels.pais.unique():
    panels_dict[p] = panels[panels.pais==p]#.dropna(axis=1,how='all')

In [51]:
panels_dict.keys()

dict_keys(['CL', 'MX', 'BR', 'PE', 'CO', 'AR'])

Where does the data go?

In [52]:
for pais, data in panels_dict.items():
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    data.to_csv(outdir+pais+"_netquest-panel.csv", encoding='utf8',index=False)

Next, bring the variables and labels in order.

In [53]:
variables = pd.read_excel(paneldir+"manual_variables.xlsx",skiprows=0)
levels = pd.read_excel(paneldir+"manual_levels.xlsx")
# this expects a the first and second sheet of netquest background export manual
# with first line (title) removed

In [54]:
print(("Variable" in variables.columns) & ("Variable" in levels.columns))

True


In [55]:
variables_dict = {}
levels_dict = {}
countryexp = re.compile("^([A-Z]{2,2})_")

for pais in panels_dict:
    all_countries = variables.Variable.apply(countryexp.findall).apply(lambda l: l[0].lower() if len(l)>0 else "None").unique()
    other_countries = [p.lower() for p in all_countries if not p==pais]
    var_df = variables[(variables.Variable.str.lower().str.startswith(pais.lower())) #country-specific var
                       | (variables.Variable.apply(lambda s: sum([s.lower().startswith(k) for k in all_countries]))==0) #not specific for another country
                       | variables.Variable.isin(["panelistAge","DESKTOP_RESPONDENT","DEVICE","DESK","TARGET",
                                                  "SOCIODEMOGRAPHICS_DATE_V0","SOCIODEMOGRAPHICS_QUEST_V0"])
                       | variables.Variable.str.contains("PNuevo")
                      ]
                       
    lev_df = levels[levels.Variable.isin(var_df.Variable)]
    
    variables_dict[pais] = var_df
    levels_dict[pais] = lev_df
    
    var_df.to_excel(outdir+pais+'_variables.xlsx',index=False)
    lev_df.to_excel(outdir+pais+'_levels.xlsx',index=False)
    

## needs supervision
Check which variables are dropped at this stage

In [56]:
list(filter(lambda x: sum([x in v[1].Variable.unique() for v in variables_dict.items()])==0, variables.Variable))

[]